In [19]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import plotly.express as px

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

# processed 10min average measurement data
#df_1m = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_cal_corr_acropolis.parquet"))
df_1h = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_cal_corr_acropolis.parquet"))


In [20]:
today = datetime(2024, 7, 1, 0, 0, 0).replace(tzinfo=timezone.utc)

dates = {
      "Example": [
            (99, datetime(2023, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 1, 1, 23, 59, 59).replace(tzinfo=timezone.utc)),
            (100, datetime(2024, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 1, 1, 23, 59, 59).replace(tzinfo=timezone.utc))
            ],
      "SMAI": [
            (1, datetime(2023, 9, 8, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 2, 5, 23, 59, 59).replace(tzinfo=timezone.utc)),
            (16, datetime(2024, 2, 8, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],  
 }

In [28]:
df_1h.head(1).collect()

system_name,sys_name_short,creation_timestamp,gmp343_raw,gmp343_compensated,gmp343_filtered,gmp343_temperature,wxt532_speed_avg,wxt532_speed_min,wxt532_speed_max,wxt532_direction_avg,wxt532_direction_min,wxt532_direction_max,wxt532_last_update_time,raspi_cpu_usage,raspi_cpu_temperature,raspi_disk_usage,enclosure_bme280_humidity,enclosure_bme280_pressure,enclosure_bme280_temperature,sht45_humidity,sht45_temperature,bme280_humidity,bme280_temperature,bme280_pressure,cal_bottle_id,cal_gmp343_raw,cal_gmp343_compensated,cal_gmp343_filtered,cal_gmp343_temperature,cal_bme280_temperature,cal_bme280_humidity,cal_bme280_pressure,cal_sht45_temperature,cal_sht45_humidity,revision,receipt_timestamp,raspi_memory_usage,wxt532_temperature,wxt532_heating_voltage,wxt532_supply_voltage,wxt532_reference_voltage,ups_battery_error_detected,ups_battery_above_voltage_threshold,ups_battery_is_fully_charged,ups_powered_by_grid,h2o_ah,h2o_v%,gmp343_dry,system_id,slope,intercept,median_bottle_1,median_bottle_2,CO2_corr,gmp343_corrected,diff,date
str,str,"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date
"""tum-esm-midcos…","""mid-cost-7""",2023-06-26 13:00:00 UTC,451.9875,499.741667,499.741667,28.766667,null,null,null,null,null,null,null,null,null,null,null,null,null,34.302917,27.8275,34.74375,27.675833,947.8975,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9.73744,1.431375,506.998869,7.0,null,null,null,null,422.672996,null,null,2023-06-26


In [29]:
# Utility

def extract_site_data(dates_site:list[tuple]):
    extracted_sites = []
    
    for date in dates_site:
        print(date[0], date[1], date[2])
        
        df_temp = df_1h.filter(pl.col("sys_name_short") == f"mid-cost-{date[0]}" ).filter(pl.col("creation_timestamp").is_between(date[1], date[2])).collect()
        
        extracted_sites.append(df_temp)
    
    df_extracted = pl.concat(extracted_sites)
    
    fig = px.line(df_extracted, x="creation_timestamp", y="gmp343_corrected", markers=True, title="CO2 Corrected [ppm]")
    fig.show()

In [30]:
extract_site_data(dates_site = dates["SMAI"])

1 2023-09-08 00:00:00+00:00 2024-02-05 23:59:59+00:00
16 2024-02-08 00:00:00+00:00 2024-07-01 00:00:00+00:00
